In [52]:
import backtrader as bt
import datetime as datetime
from backtrader_plotting import Bokeh

In [53]:
# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):
    params = (
        # 持仓够5个单位就卖出
        ('exitbars', 5),
        ('maperiod', 10),
    )

    def log(self, txt, dt=None, printLog=False):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        if printLog:
            print('%s, %s \n' % (dt.isoformat(), txt))

    def stop(self):
        self.log("(均线周期 %d)期末资金 %.2f" %
                 (self.params.maperiod, self.broker.getvalue()), printLog=True)

    def __init__(self):
        # 保存收盘价的引用
        self.dataclose = self.datas[0].close
        self.sma = bt.indicators.MovingAverageSimple(
            self.datas[0], period=self.params.maperiod)
#         bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
#         bt.indicators.WeightedMovingAverage(self.datas[0], period=25).subplot = True
#         self.rsi = bt.indicators.RSI(self.sma)
#         bt.indicators.SmoothedMovingAverage(self.rsi, period=10)
#         bt.indicators.ATR(self.datas[0]).plot = False
#         bt.indicators.StochasticSlow(self.datas[0])
#         self.macd=bt.indicators.MACDHisto(self.datas[0])

        # 跟踪挂单
        self.order = None
        # 买入价格和手续费
        self.buyprice = None
        self.buycomm = None

    # 交易状态通知，一买一卖算交易

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('交易利润, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # broker 提交/接受了，买/卖订单则什么都不做
            return

        # 检查一个订单是否完成
        # 注意: 当资金不足时，broker会拒绝订单
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    '已买入, 价格: %.2f, 费用: %.2f, 佣金 %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            elif order.issell():
                self.log('已卖出, 价格: %.2f, 费用: %.2f, 佣金 %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                self.log('已卖出, 数量: %s' %
                         (order.executed.size))

            # 记录当前交易数量
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单取消/保证金不足/拒绝')

        # 其他状态记录为：无挂起订单
        self.order = None

    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 如果有订单正在挂起，不操作
        if self.order:
            return

        # 如果没有持仓则买入
        if not self.position:
            # 今天的收盘价 < 昨天收盘价
            if self.dataclose[0] > self.sma[0]:
                # 昨天收盘价 < 前天的收盘价
                # 买入
                self.log('买入, %.2f' % self.dataclose[0])
                # 跟踪订单避免重复
                self.order = self.buy()
        else:
            # 如果已经持仓，且当前交易数据量在买入后5个单位后
            if self.dataclose[0] < self.sma[0]:
                # 全部卖出
                self.log('卖出, %.2f' % self.dataclose[0])
                # 跟踪订单避免重复
                self.order = self.sell()

In [54]:
    # 创建交易数据集
data = bt.feeds.YahooFinanceCSVData(
    dataname="orcl-1995-2014-1.csv",
    # 数据必须大于fromdate
    fromdate=datetime.datetime(2000, 1, 1),
    # 数据必须小于todate
    todate=datetime.datetime(2000, 12, 31),
    reverse=False)


In [55]:
cerebro=bt.Cerebro();
cerebro.broker.setcash(1000.0)
    # 设置佣金为0.001,除以100去掉%号
cerebro.broker.setcommission(commission=0.0)
cerebro.addsizer(bt.sizers.FixedSize,stake=10)
cerebro.adddata(data)
print("资金:%.2f" % cerebro.broker.getvalue())

资金:1000.00


In [75]:
cerebro.strats.clear()
aa=range(10, 31)
cerebro.optstrategy(TestStrategy,maperiod=aa)

cerebro.run()


2000-12-29, (均线周期 17)期末资金 949.80 

2000-12-29, (均线周期 16)期末资金 959.40 

2000-12-29, (均线周期 13)期末资金 893.90 
2000-12-29, (均线周期 15)期末资金 973.90 


2000-12-29, (均线周期 14)期末资金 896.90 

2000-12-29, (均线周期 12)期末资金 830.30 
2000-12-29, (均线周期 10)期末资金 880.30 
2000-12-29, (均线周期 11)期末资金 880.00 



2000-12-29, (均线周期 22)期末资金 1061.50 

2000-12-29, (均线周期 24)期末资金 1020.10 

2000-12-29, (均线周期 19)期末资金 1041.90 

2000-12-29, (均线周期 18)期末资金 1011.90 
2000-12-29, (均线周期 25)期末资金 1013.30 
2000-12-29, (均线周期 20)期末资金 1078.00 
2000-12-29, (均线周期 23)期末资金 1023.00 




2000-12-29, (均线周期 21)期末资金 1058.80 

2000-12-29, (均线周期 26)期末资金 998.30 

2000-12-29, (均线周期 30)期末资金 979.80 
2000-12-29, (均线周期 27)期末资金 982.20 


2000-12-29, (均线周期 28)期末资金 975.70 

2000-12-29, (均线周期 29)期末资金 983.30 



[[<backtrader.cerebro.OptReturn at 0x7f9e23fee450>],

In [57]:
cerebro.plot()

AttributeError: 'OptReturn' object has no attribute 'datas'

In [ ]:
from backtrader_plotting.schemes import Tradimo
from backtrader_plotting.bokeh.optbrowser import OptBrowser

# b = Bokeh(style='bar')
b = Bokeh(style='bar', scheme=Tradimo())
# cerebro.plot(b)

browser = OptBrowser(b, result)
browser.start()


In [ ]:
from backtrader_plotting.schemes import Tradimo
from backtrader_plotting.bokeh.optbrowser import OptBrowser

# b = Bokeh(style='bar')
b = Bokeh(style='bar', scheme=Tradimo())
cerebro.plot(b)

